## Hopfield Neural Network
### Theoretical background

The author of this neural network is John Hopfield (Nobel Prize Laureate), who studied neurons similar to the perceptrons, but with some significant differences. The essence of the problem was the use of an energy function associated with the neural network, as is common with other physical systems. The Hopfield network consists of a set of neurons that are fully interconnected in both directions.

<div style="text-align: center;">
    <img src="hopfield_net.png" alt="Hopfiled Net" style="width: 35%;">
</div>

The weights of the network are symmetric in the sense that $w_{ij} = w_{ji}$. Like a perceptron, each neuron has its threshold and a step function for activation dynamics, which is again triggered by the internal potential defined by the weighted sum of outputs from surrounding neurons. Therefore, the state of the neurons can either be standard \{0, 1\} or bipolar \{-1, +1\}. Given that the latter case is more common and has a clearer mathematical framework, we will focus on it in the following.

The main difference in the Hopfield model lies in the fact that inputs are applied to all neurons of the network in the form of values -1 and +1, followed by a cycle of gradual changes in neuron excitations until a stable state is reached. In other words, the outputs of the previous step become the new inputs of the current step. This process can be explained by the following scheme. The initial state represents the diversity of neuron excitations, which, since they are all interconnected, begin to influence each other. This can mean that one neuron tries to excite neurons unlike another, which tries to do the opposite. The result is finding a compromise - the network has relaxed into a stable state.

Let's try to express the algorithm of the Hopfield network in the following points:
1. **The weights $w_{ij}$ are defined**

$$
w_{ij} = \begin{cases} 
\sum_{s=1}^{M} x^s_i \cdot x^s_j & \text{if } i \neq j, \\
0 & \text{if } i = j,
\end{cases}
$$

where:
- $w_{ij}$ is the weight of the connection between neuron $i$ and neuron $j$,
- $x^s_i$ is the $i$-th element of the $s$-th pattern in the training set, with values in $\{ -1, +1 \}$,
- $M$ is the number of elements in the training set.

The explanation of this weight adaptation is clarified in the following image:

<div style="text-align: center;">
    <img src="adaptation.png" alt="Hopfield Net Adaptation" style="width: 35%;">
</div>

In cases b) and d), the state of excitation of both neurons is identical. This means that the new weight of the connection will be given by the relationship:

$$
w_{ij}(t+1) = w_{ij}(t) + x_i \cdot x_j = w_{ij}(t) + 1.
$$

This signifies a "strengthening" of the connection between these neurons and in the case of the network's relaxation, both neurons will strive to reach the same state. The more patterns with this state of both neurons, the greater the effort to achieve this identical state upon relaxation.

In cases a) and c), the process is reversed. Thus, the new weight of the connection will have the following value:

$$
w_{ij}(t+1) = w_{ij}(t) + x_i \cdot x_j = w_{ij}(t) - 1
$$

and the connection will evolve towards such a state that, upon relaxation of the network, the states of both neurons will be different.

2. **Initialization with an unknown pattern**
$$
\mu_i(0) = x_i, \quad 1 \leq i \leq N
$$

where
  - $\mu_i(t)$ denotes the state of excitation of neuron $i$ at time $t$,
  - $N$ represents the number of neurons in the network.

3. **Iterate until a stable state is reached**
$$
\mu_i(t+1) = \text{sgn\{+1,-1\}}\left(\sum_{j=1}^{N} w_{ij} \mu_j(t) \right), \quad 1 \leq i \leq N,
$$

4. **Continue until** 
$$
\mu_i(t+1) = \mu_i(t), \quad 1 \leq i \leq N
$$

#### Energy Function
The principle of encoding and subsequently retrieving patterns can best be explained through the energy function of a Hopfield network. Let us imagine that this energy landscape has its local minima representing the patterns presented during the network's adaptation phase. The subsequent input defines a point on this energy landscape. The iterative process of the network's relaxation describes the movement of this point across the energy landscape towards one of the stable points—an attractor—represented by one of its local minima.

The above requirements for representing patterns of the training set in the form of local energy minima and their discovery during the relaxation process correspond to a function of the following form (for simplicity, we will consider the threshold values of neurons to be zero in subsequent discussions):

The energy \( E \) of the Hopfield network is given by:

$$
E = -\sum_{i} \sum_{j \neq i} w_{ij} x_i x_j,
$$

where:
- $w_{ij}$ is the weight of the connection between node $i$ and $j$,
- $x_i$ is the state of neuron $i$.

A closer examination of the aforementioned energy function of the network yields the following conclusions:
The inner sum actually represents the internal potential of neuron $i$. If its sign differs from the state of excitation of the neuron (which is actually an error state), the energy value is higher than in a correct situation where the potential and excitation of the neuron have the same sign. The more such "mismatches" there are in the neural network, the greater its energy.

### Implemenation in Python

**Import modules**

In [1]:
import numpy as np
from hopfield_net import HopfieldNet
from data_set import DataSet

**Create training set for the patterns 1 a 2**

In [2]:
patterns = [
    [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    ],
    [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0],
    [0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0],
    ],
]

training_set = DataSet(8, 7)
for pattern in patterns:
        training_set.add_pattern(pattern)
training_set.display_patterns()


+-------+
|       |
|   *   |
|  **   |
| * *   |
|   *   |
|   *   |
|   *   |
|       |
+-------+
+-------+
|       |
|  ***  |
| *   * |
|    *  |
|   *   |
|  *    |
| ***** |
|       |
+-------+


**Train the Hopfield Net for these two patterns**

In [3]:
hopfield_net = HopfieldNet(training_set.num_neurons)
hopfield_net.train(training_set.patterns)

**Define input data set with two noisy inputs**

In [4]:
inputs = [
    [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 1],
    [0, 0, 1, 1, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
    ],
    [
    [1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 1, 0, 0],
    [1, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 1, 1, 1, 1, 0],
    [0, 0, 1, 0, 1, 0, 0],
    ],
]

input_set = DataSet(8, 7)
for input_pattern in inputs:
    input_set.add_pattern(input_pattern)
input_set.display_patterns()

+-------+
|       |
| * *  *|
|  **   |
| * *   |
|   **  |
|   *   |
|    *  |
|      *|
+-------+
+-------+
|*      |
|*   *  |
|**   * |
|    ** |
|   *   |
|  * *  |
| ***** |
|  * *  |
+-------+


**Searching for the nearest pattern**

Hopfield Net finds the local minima of its energy through relaxation algorithm.

In [5]:
result_set = DataSet(8, 7)
for i in range(len(input_set.patterns)):
    result = hopfield_net.run(input_set.patterns[i], max_cycles=10)
    result_set.patterns.append(result)
result_set.display_patterns()

+-------+
|       |
|   *   |
|  **   |
| * *   |
|   *   |
|   *   |
|   *   |
|       |
+-------+
+-------+
|       |
|  ***  |
| *   * |
|    *  |
|   *   |
|  *    |
| ***** |
|       |
+-------+


Original patterns were recovered. 

What would happen if additional pattern was introduced to Hopfield net?

In [6]:
training_set.add_pattern([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0],
    [0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1, 0],
    [0, 0, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
])

training_set.display_patterns()

+-------+
|       |
|   *   |
|  **   |
| * *   |
|   *   |
|   *   |
|   *   |
|       |
+-------+
+-------+
|       |
|  ***  |
| *   * |
|    *  |
|   *   |
|  *    |
| ***** |
|       |
+-------+
+-------+
|       |
|  ***  |
| *   * |
|   **  |
|     * |
| *   * |
|  ***  |
|       |
+-------+


Train Hopfiled Net with this extended and try to search for patterns with the original input set.

In [7]:
hopfield_net = HopfieldNet(training_set.num_neurons)
hopfield_net.train(training_set.patterns)

result_set = DataSet(8, 7)
for i in range(len(input_set.patterns)):
    result = hopfield_net.run(input_set.patterns[i], max_cycles=10)
    result_set.patterns.append(result)
result_set.display_patterns()

+-------+
|       |
|  ***  |
| *   * |
| * **  |
|   *   |
|       |
|  ***  |
|       |
+-------+
+-------+
|       |
|  ***  |
| *   * |
|   **  |
|   *   |
|       |
|  ***  |
|       |
+-------+


Result shows that none of original encoded patterns was found. Let's try to use original training ser as an input set.

In [8]:
result_set = DataSet(8, 7)
for i in range(len(training_set.patterns)):
    result = hopfield_net.run(training_set.patterns[i], max_cycles=10)
    result_set.patterns.append(result)
result_set.display_patterns()

+-------+
|       |
|   *   |
|  **   |
| * *   |
|   *   |
|   *   |
|   *   |
|       |
+-------+
+-------+
|       |
|  ***  |
| *   * |
|   **  |
|   *   |
|       |
|  ***  |
|       |
+-------+
+-------+
|       |
|  ***  |
| *   * |
|   **  |
|   *   |
|       |
|  ***  |
|       |
+-------+


Original pattern 1 was found but patterns 2 and 3 created one common local energy minimum that does not correspond to the original patterns. Phantom appeared in neural network because 2 a 3 are close to each other. 

This issue of overloading Hopfield net can be solved by distribution of patterns in the input space.

In [9]:
patterns = [
    [
    [0, 0, 1, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    ],
    [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0],
    [0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0],
    ],
    [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 0],
    [0, 0, 1, 0, 0, 0, 1],
    [0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 0, 1, 0, 0, 0, 1],
    [0, 0, 0, 1, 1, 1, 0],
    ]
]

training_set = DataSet(8, 7)
for pattern in patterns:
        training_set.add_pattern(pattern)
training_set.display_patterns()

+-------+
|  *    |
| **    |
|* *    |
|  *    |
|  *    |
|  *    |
|       |
|       |
+-------+
+-------+
|       |
|  ***  |
| *   * |
|    *  |
|   *   |
|  *    |
| ***** |
|       |
+-------+
+-------+
|       |
|       |
|   *** |
|  *   *|
|    ** |
|      *|
|  *   *|
|   *** |
+-------+


**Create new net and train it with new data.**

In [10]:
hopfield_net = HopfieldNet(training_set.num_neurons)
hopfield_net.train(training_set.patterns)

**Define new input data set with three noisy inputs**

In [11]:
inputs = [
    [
    [1, 0, 1, 0, 0, 1, 0],
    [0, 1, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0],
    ],
    [
    [1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 1, 0, 0],
    [1, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 1, 1, 1, 1, 0],
    [0, 0, 1, 0, 1, 0, 0],
    ],
    [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 1, 0],
    [0, 0, 0, 1, 1, 1, 0],
    [0, 0, 1, 0, 0, 0, 1],
    [0, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 1, 0],
    ]
]

input_set = DataSet(8, 7)
for input_pattern in inputs:
    input_set.add_pattern(input_pattern)
input_set.display_patterns()

+-------+
|* *  * |
| **    |
|* *    |
|  * *  |
|  *    |
|   *   |
|     * |
|       |
+-------+
+-------+
|*      |
|*   *  |
|**   * |
|    ** |
|   *   |
|  * *  |
| ***** |
|  * *  |
+-------+
+-------+
|       |
| **  * |
|   *** |
|  *   *|
|    ** |
| *    *|
|  *   *|
|  ** * |
+-------+


**Searching for the nearest pattern**

Hopfield Net finds the local minima of its energy through relaxation algorithm.

In [12]:
result_set = DataSet(8, 7)
for i in range(len(input_set.patterns)):
    result = hopfield_net.run(input_set.patterns[i], max_cycles=10)
    result_set.patterns.append(result)
result_set.display_patterns()

+-------+
|  *    |
| **    |
|* *    |
|  *    |
|  *    |
|  *    |
|       |
|       |
+-------+
+-------+
|       |
|  ***  |
| *   * |
|    *  |
|   *   |
|  *    |
| ***** |
|       |
+-------+
+-------+
|       |
|       |
|   *** |
|  *   *|
|    ** |
|      *|
|  *   *|
|   *** |
+-------+


All required patterns were found. 👍

## Summary

Hopfield networks, a type of recurrent artificial neural network, have been widely studied and applied, particularly for their ability to act as content-addressable (“associative”) memory systems. Despite their interesting properties and applications, there are several limitations associated with Hopfield nets:

* **Limited Capacity**: Hopfield networks can store only a limited number of patterns. The general rule of thumb is that a Hopfield network can store about **0.15 times the number of neurons** before the network becomes saturated and retrieval accuracy diminishes. This limitation arises from the nature of their energy landscape, where adding too many patterns can create confusing overlaps in the state space.
* **Spurious States**: Besides the desired attractor states (the memories), Hopfield networks can have spurious or undesired attractor states. These are states not explicitly stored but arise from the combinations of the stored patterns. These spurious states can lead to incorrect retrieval of memories.
* **Binary Units**: Traditional Hopfield networks use binary threshold units, which can limit their applicability to real-world problems that may require continuous or multi-level responses. While variations exist that use continuous values, they are not as widely analyzed or understood.
* **Convergence to Local Minima**: Hopfield networks are susceptible to converging to local minima rather than the global minimum in their energy landscape. This issue means that the network may settle on a suboptimal pattern that is not the closest match to the input if that pattern has a stronger attraction or if the input is closer to a local minimum.

These limitations underscore the importance of understanding the specific characteristics and requirements of a problem before choosing Hopfield networks as a solution. For many modern applications, alternative neural network architectures may offer more flexibility and capacity.

